In the second cut at this, I'm more comfortable with aligning directly with Wikidata in certain areas. This new version of the notebook runs essentially the same query in both the GeoKB and Wikidata, ensuring we have our much simpler representation of a select few properties in the GeoKB and adding same as relationships with corresponding Wikidata items.

Rather than rely strictly on how items are type-classified through instance of claims, we can simply look for all items that have ISO3166 identifiers in both Wikidata and the GeoKB. These are the unique international alpha and numeric identifiers for entities that are the "countries" we care about building linkages. Some of those are like the U.S. Territories in that they are not exactly a sovereign nation-state. In the initial work to organize these items in the GeoKB, I had to classify them separately as "country" or "U.S. Territory." At this stage, I'm just updating a couple of things:

* Adding same as relationship to corresponding Wikidata items
* Adding flag symbols to aliases (just because)
* Using Wikidata descriptions (after reviewing them to make sure there wasn't anything weird at this point in time)

In [1]:
from wbmaker import WikibaseConnection
import pandas as pd
import re

geokb = WikibaseConnection("GEOKB_CLOUD")

In [2]:
def aggregator(x):
    return str(x.unique()[0]) if len(x.unique()) == 1 else list(sorted(x.unique()))

def extract_flags(aliases):
    flag_regex = re.compile(r'([\U0001F1E6-\U0001F1FF]{2})')
    return flag_regex.findall(aliases)

geokb_country_query = """
PREFIX wdt: <https://geokb.wikibase.cloud/prop/direct/>

SELECT ?country ?countryLabel ?countryAltLabel ?countryDescription
?iso3166_alpha2 ?iso3166_alpha3 ?iso3166_numeric
WHERE {
  ?country wdt:P1 ?instance_of ;
           wdt:P10 ?iso3166_alpha2 ;
           wdt:P38 ?iso3166_alpha3 ;
           wdt:P37 ?iso3166_numeric .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
"""

geokb_countries = geokb.sparql_query(geokb_country_query)
geokb_countries = geokb_countries.groupby('country', as_index=False).agg(aggregator)
geokb_countries['qid'] = geokb_countries['country'].apply(lambda x: x.split('/')[-1])

wd_country_query = """
SELECT ?country ?countryLabel ?countryAltLabel ?countryDescription
?iso3166_alpha2 ?iso3166_alpha3 ?iso3166_numeric
WHERE {
  ?country wdt:P31 ?instance_of ;
           wdt:P297 ?iso3166_alpha2 ;
           wdt:P298 ?iso3166_alpha3 ;
           wdt:P299 ?iso3166_numeric .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
"""

wd_countries = geokb.sparql_query(
    query=wd_country_query,
    endpoint="https://query.wikidata.org/sparql"
)
wd_countries = wd_countries.groupby('country', as_index=False).agg(aggregator)
wd_countries['flags'] = wd_countries['countryAltLabel'].apply(extract_flags)

In [3]:
merged_country_items = pd.merge(
    left=geokb_countries[['qid','iso3166_alpha2','countryAltLabel']],
    right=wd_countries[['iso3166_alpha2','country','countryDescription','flags']].rename(columns={'country':'wd_entity', 'countryDescription': 'wd_description'}),
    how='left',
    on='iso3166_alpha2'
).convert_dtypes().reset_index(drop=True)

merged_country_items.head()


,qid,iso3166_alpha2,countryAltLabel,wd_entity,wd_description,flags
0,Q100,AE,"AE, ARE, 🇦🇪",http://www.wikidata.org/entity/Q878,sovereign state in Southwest Asia,[🇦🇪]
1,Q101,VN,"Vietnam, VN, Viet Nam, VNM, 🇻🇳",http://www.wikidata.org/entity/Q881,country in Southeast Asia,[🇻🇳]
2,Q102,KR,"KR, South Korea, KOR, 🇰🇷",http://www.wikidata.org/entity/Q884,country in East Asia,[🇰🇷]
3,Q103,AF,"AF, Afghanistan, AFG, 🇦🇫",http://www.wikidata.org/entity/Q889,country in Central and South Asia,[🇦🇫]
4,Q104,BD,"Bangladesh, BD, BGD, 🇧🇩",http://www.wikidata.org/entity/Q902,country in South Asia,[🇧🇩]


In [5]:
for _, row in merged_country_items.iterrows():
    item = geokb.wbi.item.get(row['qid'])
    aliases = [i.strip() for i in row['countryAltLabel'].split(',')]
    aliases.extend(row['flags'])

    if row['flags']:
        item.aliases.set('en', aliases, action_if_exists=geokb.action_if_exists.REPLACE_ALL)

    item.descriptions.set('en', row['wd_description'])
    
    item.claims.add(
        geokb.datatypes.URL(
            prop_nr=geokb.prop_lookup['same as'],
            value=row['wd_entity']
        )
    )

    response = item.write(
        summary="Added country flag to aliases and same as linkage to Wikidata"
    )
    print(response.id)

Q100
Q101
Q102
Q103
Q104
Q105
Q106
Q107
Q108
Q109
Q110
Q111
Q112
Q113
Q114
Q115
Q116
Q117
Q118
Q119
Q120
Q121
Q122
Q123
Q124
Q125
Q126
Q127
Q128
Q129
Q130
Q131
Q132
Q133
Q134
Q135
Q136
Q137
Q138
Q139
Q140
Q141
Q142
Q143
Q144
Q145
Q146
Q147
Q148
Q149
Q150
Q153
Q154
Q155
Q157
Q158
Q159
Q160
Q161
Q162
Q163
Q164
Q165
Q166
Q167
Q168
Q169
Q170
Q171
Q172
Q173
Q174
Q175
Q176
Q177
Q178
Q179
Q180
Q181
Q182
Q183
Q184
Q185
Q186
Q187
Q188
Q189
Q190
Q191
Q192
Q193
Q194
Q195
Q196
Q197
Q198
Q199
Q200
Q201
Q202
Q203
Q204
Q205
Q206
Q26696
Q26697
Q26698
Q26699
Q26701
Q26702
Q26703
Q26705
Q26706
Q26707
Q26708
Q26709
Q26710
Q26711
Q26712
Q26713
Q26714
Q26715
Q26716
Q26717
Q26718
Q26719
Q26720
Q26721
Q26722
Q26723
Q26724
Q26725
Q26726
Q26727
Q26728
Q26729
Q26730
Q26732
Q26733
Q26734
Q26735
Q26736
Q26737
Q26738
Q26739
Q26740
Q26741
Q26742
Q26743
Q26744
Q26745
Q26746
Q26747
Q26748
Q26749
Q26750
Q26751
Q26752
Q26754
Q26755
Q26756
Q26757
Q26758
Q26759
Q26760
Q26761
Q26762
Q26763
Q26764
Q26765
Q26766
Q26767
Q267